In [1]:
# Parameters
x_encoding = [
    {"name": "prev_contributions", "n_levels": 21, "encoding": "numeric"},
    {"name": "prev_punishments", "n_levels": 31, "encoding": "numeric"},
    {"name": "round_number", "n_levels": 16, "encoding": "numeric"},
    {"name": "prev_common_good", "norm": 128, "etype": "float"},
    {"name": "prev_valid", "etype": "bool"},
]
n_contributions = 21
n_punishments = 31
n_cross_val = 2
fraction_training = 1.0
data_file = "../../data/experiments/pilot_random1_player_round_slim.csv"
output_path = "../../data/training/dev"
labels = {}
model_name = "mlp"
model_args = {"n_layers": 2, "hidden_size": 40}
optimizer_args = {"lr": 0.0001, "weight_decay": 1e-05}
train_args = {"epochs": 1000, "batch_size": 40, "clamp_grad": 1, "eval_period": 10}
device = "cpu"


In [2]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import torch as th
from aimanager.generic.data import create_syn_data, create_torch_data, get_cross_validations
from aimanager.artificial_humans import AH_MODELS
from aimanager.artificial_humans.evaluation import Evaluator
from aimanager.utils.array_to_df import using_multiindex
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader

output_path = os.path.join(output_path, 'data')

/Users/levinbrinkmann/repros/algorithmic-institutions/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv(data_file)


data = create_torch_data(df)
syn_data = create_syn_data(n_contribution=21, n_punishment=31)

In [4]:
th_device = th.device(device)

metrics = []
confusion_matrix = []
syn_pred = []
ev = Evaluator()

th_device = th.device(device)

syn_index = ['prev_punishments', 'prev_contributions']

def create_fully_connected(n_nodes):
    return th.tensor([[i,j]
        for i in range(n_nodes)
        for j in range(n_nodes)
    ]).T

def encode(model, data, *, mask=True, index=False, x_encode=True, y_encode=True, u_encode=False, device, n_player=4):
    data = {
        'mask': data['valid'] if mask else None,
        'x': model.x_encoder(**data) if x_encode else None,
        'y_enc': model.y_encoder(**data) if y_encode else None,
        'y': data['contributions'] if y_encode else None,
        'u': model.u_encoder(**data) if u_encode else None,
        'info': th.stack([data[c] for c in syn_index], dim=-1) if index else None,
    }
    data = {
        k: v.to(device)
        for k, v in data.items()
        if v is not None
    }

    edge_attr = th.zeros(n_player*n_player,0)
    edge_index = create_fully_connected(n_player)

    n_episodes = list(data.values())[0].shape[0]
    dataset = [
        Data(**{k: v[i] for k, v in data.items()}, edge_attr=edge_attr, edge_index=edge_index, idx=i, group_idx=i, num_nodes=n_player)
        for i in range(n_episodes)
    ]
    return dataset


for i, (train_data, test_data) in enumerate(get_cross_validations(data, n_cross_val, fraction_training)):
    model = AH_MODELS[model_name](
        n_contributions=n_contributions, n_punishments=n_punishments, x_encoding=x_encoding,
        **model_args).to(th_device)

    train_data_ = encode(model, train_data, mask=True, device=th_device)
    test_data_ = encode(model, test_data, mask=True, device=th_device)
    syn_data_ = encode(model, syn_data, mask=False, y_encode=False, index=True, device=th_device)

    print(len(train_data_))

    syn_df = using_multiindex(
        Batch.from_data_list(syn_data_)['info'], ['idx', 'round_number'], syn_index)

    ev.set_data(test=test_data_, train=train_data_, syn=syn_data_, syn_df=syn_df)

    optimizer = th.optim.Adam(model.parameters(), **optimizer_args)
    loss_fn = th.nn.CrossEntropyLoss(reduction='none')
    sum_loss = 0
    n_steps = 0

    for e in range(train_args['epochs']):
        ev.set_labels(cv_split=i, epoch=e)
        model.train()
        for j, batch_data in enumerate(iter(DataLoader(train_data_, shuffle=True, batch_size=train_args['batch_size']))):

            optimizer.zero_grad()
            py = model(batch_data).flatten(end_dim=-2)
            y_true = batch_data['y_enc'].flatten(end_dim=-2)
            mask = batch_data['mask'].flatten()
            loss = loss_fn(py, y_true)
            loss = (loss * mask).sum() / mask.sum()

            loss.backward()

            if train_args['clamp_grad']:
                for param in model.parameters():
                    param.grad.data.clamp_(-train_args['clamp_grad'], train_args['clamp_grad'])
            optimizer.step()
            sum_loss += loss.item()
            n_steps +=1
        
        if e % train_args['eval_period'] == 0:
            avg_loss = sum_loss/n_steps
            print(f'CV {i} | Epoch {e} | Loss {avg_loss}')
            ev.add_loss(avg_loss)
            ev.eval_set(model, 'train')
            ev.eval_set(model, 'test')
            sum_loss = 0
            n_steps = 0

    ev.eval_sync(model, syn_index=syn_index)

ev.save(output_path, labels)
model_path = os.path.join(output_path, 'model.pt')
model.save(model_path)

135
CV 0 | Epoch 0 | Loss 3.0452667474746704
CV 0 | Epoch 10 | Loss 3.0334501385688784
CV 0 | Epoch 20 | Loss 3.0111855030059815
CV 0 | Epoch 30 | Loss 2.9892133712768554
CV 0 | Epoch 40 | Loss 2.9658118963241575
CV 0 | Epoch 50 | Loss 2.943079912662506
CV 0 | Epoch 60 | Loss 2.914994889497757
CV 0 | Epoch 70 | Loss 2.890031671524048
CV 0 | Epoch 80 | Loss 2.8635134518146517
CV 0 | Epoch 90 | Loss 2.8340967893600464
CV 0 | Epoch 100 | Loss 2.8059243977069857
CV 0 | Epoch 110 | Loss 2.781332176923752
CV 0 | Epoch 120 | Loss 2.7482638239860533
CV 0 | Epoch 130 | Loss 2.7195549964904786
CV 0 | Epoch 140 | Loss 2.6989544868469237
CV 0 | Epoch 150 | Loss 2.6694578886032105
CV 0 | Epoch 160 | Loss 2.6373047828674316
CV 0 | Epoch 170 | Loss 2.6215018212795256
CV 0 | Epoch 180 | Loss 2.5833326458930967
CV 0 | Epoch 190 | Loss 2.5672889590263366
CV 0 | Epoch 200 | Loss 2.5632621884346007
CV 0 | Epoch 210 | Loss 2.5359054028987886
CV 0 | Epoch 220 | Loss 2.5300695300102234
CV 0 | Epoch 230 | Los